In [18]:
from faker import Faker
import pandas as pd
from datetime import datetime

fake = Faker(random_state=24)
fecha_minima = datetime.strptime('2006-11-15', '%Y-%m-%d') # En la página dice 15 de Noviembre del 2006 -> Reporte del 2023-2024 al final del PDF
fecha_maxima = datetime.strptime('2024-04-01', '%Y-%m-%d')

amount_predefined = [10, 15, 35, 50]
amount_free = [1, 5, 20, 70, 90, 100] 
amount_choices = amount_predefined + amount_free # Combinar ambas listas

data2 = []

# Generar nombres ficticios
for _ in range(1000):
    name = fake.name()
    try:
        first_name, last_name = name.split()
    except ValueError:
        first_name = name
        last_name = ""

    # Utilizar los valores de first_name y last_name para crear la dirección de correo electrónico
    email = f"{first_name.lower()}.{last_name.lower()}@gmail.com"

    date = fake.date_between_dates(date_start= fecha_minima, date_end=fecha_maxima) # Fecha de benefactor
    is_partner = fake.random_element([0, 1]) # is_partner -> 0: donante_individual - 1: empresa (partner)
    donation_frecuency = fake.random_element([0, 1])  # Mensual es 1 frecuencia_de_pago

    # - Si is_partner es 1, suscription_status se establece en 1. 
    # - Si is_partner es 0, suscription_status se asigna aleatoriamente entre 0 y 1, ya que se trata de un donante individual. 
    # Sin embargo, aquellos que originalmente quedaron como 1 en is_partner tendrán suscription_status fijo en 1.

    if is_partner == 1:
      suscription_status = 1
      company = fake.random_element(['Pepsi', 'Amazon', 'Jonhsons', 'Nasscom', 'Vodafone', 'Baxter', 'The Oberoi', 'CBN', 'Hyatt', 'Westin', 'YCI', 'CaSa', 'Cromax', 'Nike', 'Cosco', 'Indigo']) # company
      role_company = fake.random_element(['CEO',' CTO', 'Employee']) # role
      donation_type = fake.random_element([1, 2, 3, 4])  # Si es partner, se puede elegir cualquier tipo de donación -> 1: Dinero - 2: Comida - 3: Material - 4: Otro
      if donation_type == 1:
          amount = fake.random_int(min=200, max=2000)
          method_pay = fake.random_element([1,2,3])
      else:
          amount = 0
          if (company == 'Amazon'):
              donation_type = 2
             
    else:
      suscription_status = 1 if donation_frecuency == 1 else 0
      company = 'None' # -> Si coloco un 0 y se quiere aplicar un label encoder, se van a pisar los valores 0 de is_partner == 1 and is_partner == 0
      role_company = 'None'
      donation_type = 1  # Si no es partner, el tipo de donación siempre es 1 = 'Dinero'
      amount = fake.random_element(amount_choices)        
      method_pay = fake.random_element([1,2,3]) # 1: Tarjeta - 2 :  Paypal  - 3:  Googlepay
   
    donation_date = fake.date_between_dates(date_start= fecha_minima, date_end=fecha_maxima) #Fecha de la donación
    data2.append([name, email, date, is_partner, company, role_company, donation_frecuency, suscription_status, donation_type, amount, method_pay, donation_date])


df2 = pd.DataFrame(data2, columns=['name', 'email', 'date', 'is_partner',  'company', 'role_company',  'donation_frecuency', 'suscription_status', 'donation_type', 'amount', 'method_pay','donation_date'])
df2

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
0,Christopher May,christopher.may@gmail.com,2012-11-13,1,Pepsi,Employee,1,1,4,0,2,2015-06-03
1,Joseph Monroe,joseph.monroe@gmail.com,2018-07-29,1,Hyatt,CTO,0,1,3,0,2,2007-02-23
2,Amy West,amy.west@gmail.com,2018-01-15,0,None,None,1,1,1,15,3,2010-08-19
3,Nicholas Lee,nicholas.lee@gmail.com,2016-03-19,1,Indigo,CTO,0,1,2,0,3,2010-04-06
4,Robert Jimenez,robert.jimenez@gmail.com,2022-07-16,1,Jonhsons,CTO,1,1,1,520,3,2023-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Brian Garrett,brian.garrett@gmail.com,2024-03-19,1,The Oberoi,CEO,1,1,4,0,1,2018-05-13
996,Kayla Weeks,kayla.weeks@gmail.com,2014-01-08,0,None,None,1,1,1,1,2,2019-03-16
997,Nathan Smith,nathan.smith@gmail.com,2017-02-22,1,Vodafone,CTO,0,1,1,616,2,2011-02-02
998,Jacob Grimes,jacob.grimes@gmail.com,2015-04-10,0,None,None,0,0,1,70,1,2016-11-17


In [20]:
from dateutil.relativedelta import relativedelta
import random 
# Primero, seleccionamos los registros que cumplen las condiciones
filtered_df = df2[(df2['is_partner'] == 0) & (df2['donation_frecuency'] == 1)]

# Luego, creamos una lista para almacenar los registros duplicados con modificaciones
duplicated_records = []

# Iteramos sobre los registros seleccionados y creamos duplicados con modificaciones
for _, row in filtered_df.iterrows():
    # Generamos un valor aleatorio para la cantidad de duplicados
    num_duplicates = random.randint(2, 24)  # Por ejemplo, generamos un número aleatorio entre 1 y 10
    for i in range(num_duplicates):
        duplicated_row = row.copy()
        duplicated_row['donation_date'] += relativedelta(months=i)
        duplicated_records.append(duplicated_row)
            

# El primer ciclo for itera sobre cada fila (registro) en el DataFrame filtered_df, 
# que contiene solo las filas que cumplen con las condiciones especificadas ((df2['is_partner'] == 0) & (df2['donation_frecuency'] == 1)).

# El segundo ciclo for itera sobre los valores en la lista num_duplicates_list. Esta lista contiene la cantidad de duplicados que se deben crear 
# para cada fila seleccionada en el primer ciclo for.

# El tercer ciclo for, dentro del segundo ciclo, itera num_duplicates veces. num_duplicates es el valor actual de la lista num_duplicates_list 
# en cada iteración del segundo ciclo. Este ciclo se utiliza para crear múltiples duplicados de la misma fila.

# Convertimos la lista de registros duplicados en un DataFrame
duplicated_df = pd.DataFrame(duplicated_records)

# Concatenamos el DataFrame duplicado con el DataFrame original
result_df = pd.concat([df2, duplicated_df], ignore_index=True)
result_df.head()

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
0,Christopher May,christopher.may@gmail.com,2012-11-13,1,Pepsi,Employee,1,1,4,0,2,2015-06-03
1,Joseph Monroe,joseph.monroe@gmail.com,2018-07-29,1,Hyatt,CTO,0,1,3,0,2,2007-02-23
2,Amy West,amy.west@gmail.com,2018-01-15,0,None,None,1,1,1,15,3,2010-08-19
3,Nicholas Lee,nicholas.lee@gmail.com,2016-03-19,1,Indigo,CTO,0,1,2,0,3,2010-04-06
4,Robert Jimenez,robert.jimenez@gmail.com,2022-07-16,1,Jonhsons,CTO,1,1,1,520,3,2023-08-18


In [23]:
result_df.shape

(3825, 12)

In [21]:
result_df.to_csv('datos_donantes.csv', index=False)